# Dark analysis

The input consists of sets of 31 dark frames and 31 bias frames. One set was taken with the same settings used for the light exposures; the other set with settings identical with the flat exposures:

 - Darks and bias for the ligth exposures: ISO 6400 and 12800, exposure time 3.2s
 - Darks and bias for the flat exposures: ISO 100, exposure 1/40s

Bias in both sets were taken at 1/8000s

In [ ]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

import exifread
import rawpy

## Utility functions 

In [ ]:
# Stats
def stats(array, x, y, size, name, results):
    slice_x = slice(x,x+100)
    slice_y = slice(y,y+100)

    mean = np.mean(array[slice_x,slice_y])
    std = np.std(array[slice_x,slice_y])
    
    results[name] = [mean, std]
    print(name, "  mean =", mean, "stddev =", std)

In [ ]:
# Combine darks from list, using a provided numpy averaging function.
def combine_arrays(file_list, combination_function=np.average):
    stack = None
    for fname, i in zip(file_list, range(len(file_list))):

        raw = rawpy.imread(fname)
        array = raw.raw_image

        if stack is None:            
            stack = np.zeros(shape=(array.shape[0],array.shape[1],len(file_list)), dtype=float)

        # Actual raw DN values are obtained by subtracting the
        # camera-created black level per channel. In the Sony A7, this
        # parameter is the same for all channels, and constant.
        # Just subtract it from everything.
        stack[:,:,i] = np.array(array, dtype=float) - 512.

    return combination_function(stack, axis=2)

In [ ]:
def process(path, title, results, vmin=-8, vmax=4, combination_function=np.average):
    list_p = glob.glob(path + '/*.ARW')

    # get exposure time
    f = open(list_p[0], 'rb')
    tags = exifread.process_file(f)
    exptime = str(tags['EXIF ExposureTime'])    
    
    combined_array = combine_arrays(list_p, combination_function=combination_function)
    
    results[title] = {}
    size = 1000

    x = int(combined_array.shape[0] / 2)
    y = int(combined_array.shape[1] / 2)
    stats(combined_array, x, y, size, "Center:", results[title])

    x = 10
    y = 10
    stats(combined_array, x, y, size, "Corner:", results[title])

    plt.figure(figsize=[10, 6])
    plt.imshow(combined_array, vmin=vmin, vmax=vmax)
    plt.colorbar()
    plt.title(title + " " + exptime)

## Define paths to data files

In [ ]:
path1 = '../astrophotography_data/MilkyWayPrettyBoy/12800/'
path2 = '../astrophotography_data/MilkyWayPrettyBoy/6400/'
path3 = '../astrophotography_data/MilkyWayPrettyBoy/darks/ISO100/'

## Define dictionary to store summary of results

In [ ]:
results = {}

## Analysis of dark/bias for light exposures

In [ ]:
dark_path = os.path.join(path1,'dark')
process(dark_path, 'Average dark - ISO 12800 31 frames', results)

In [ ]:
bias_path = os.path.join(path1,'bias')
process(bias_path, 'Average bias - ISO 12800 31 frames', results)

In [ ]:
dark_path = os.path.join(path2,'dark')
process(dark_path, 'Average dark - ISO 6400 31 frames', results)

In [ ]:
bias_path = os.path.join(path2,'bias')
process(bias_path, 'Average bias - ISO 6400 31 frames', results)

In [ ]:
bias_path = os.path.join(path1,'bias')
process(bias_path, 'Bias standard deviation - ISO 12800 31 frames', results,
        vmin=9, vmax=14, combination_function=np.std)

In [ ]:
bias_path = os.path.join(path2,'bias')
process(bias_path, 'Bias standard deviation - ISO 6400 31 frames', results,
        vmin=5, vmax=8, combination_function=np.std)

## Analysis of dark/bias for flat exposures (ISO 100)

In [ ]:
dark_path = os.path.join(path3,'dark')
process(dark_path, 'Average dark - ISO 100 31 frames', results,
       vmin=0.6, vmax=1.4)

In [ ]:
bias_path = os.path.join(path3,'bias')
process(bias_path, 'Average bias - ISO 100 31 frames', results,
       vmin=0.8, vmax=1.8,)

In [ ]:
bias_path = os.path.join(path3,'bias')
process(bias_path, 'Bias standard deviation - ISO 100 31 frames', results,
        vmin=2.0, vmax=2.4, combination_function=np.std)

In [ ]:
for k1 in results:
    print(k1)
    r = results[k1]
    for k2 in r:
        print(k2, r[k2])
    print()

## Conclusions

Bias variance for any given pixel, along a sequence of bias exposures:
  - ISO   100 - 2.2 DN
  - ISO  6400 - 6.3 DN
  - ISO 12800 -  12 DN
 
These are large when compared with the across-detector bias variance in a single frame - thus should be a good estimator of the sensor readnoise.

Darks have low mean values at any ISO, but the variance increases rapidily with increasing ISO. 
